## Paso 01 - Lectura del conjunto de información

In [ ]:
# Importar las librerías de que nos ayudarán a obtener la información, organizarla y procesarla
import pandas as pd

In [ ]:
# Lectura del archivo que contiene el corpus con la información clasificada
# y preprocesada (Ham VS Spam)
df2 = pd.read_csv('Spamless_DataSet_N.csv')
df2.head(20)

# Referencia para la lectura de archivos en: https://codeday.me/es/qa/20190412/477724.html

## Paso 02 - Verificación y validación de la información

In [ ]:
# Para detectar si tenemos filas vacías
print("Tamaño del data frame: " + str(len(df2)))
print("Cantidad de filas vacías:")
print(df2.isnull().sum())

# Eliminamos las filas vacías y vemos el tamaño del nuevo df
df2 = df2.dropna()
df2 = df2[df2['mensaje'] != ""]
print("\nTamaño del data frame sin vacíos: " + str(len(df2)))

In [ ]:
# Verificar cuales son las clasificaciones que tenemos en nuestro DataSet
print("Clases que tenemos en el DataSet:")
print(df2['clase'].unique())

# Revisamos la cantidad de ejemplos que tenemos por cada clase
print("\nCantidad de ejemplos que tenemos por clase:")
print(df2['clase'].value_counts())

In [ ]:
# Ploteamos la cantidad de ejemplos que tenemos de cada clase
# para ver si esta se encuentra balanceada o no
import matplotlib.pyplot as plt

Totales = df2['clase'].value_counts()
plt.bar(['Spam', 'NoSpam'], Totales)
plt.xticks(rotation = 45)
plt.title('Cantidad de ejemplos de cada clase')
plt.show()

## Paso 04 - Buscar relaciones en la información

In [ ]:
# Para analizar la información, ploteamos un histograma
# que nos indica la frecuencia de caracteres tanto en
# Ham como en Spam para revisar su comportamiento

Spams = []
NoSpams = []
for clase, mensaje in zip(df2['clase'], df2['mensaje']):
    if clase == 'Spam':
        Spams.append(len(mensaje))
    else:
        NoSpams.append(len(mensaje))


plt.hist(Spams, bins=15)
plt.hist(NoSpams, bins=15)
plt.legend(['Spam', 'No Spam'])
plt.xlabel('Cantidad de Caracteres en los ejemplos')
plt.ylabel('Total de ejemplos')
plt.show()

In [ ]:
# Importamos el núcleo de trabajo para hacer la tokenización
import spacy.cli
spacy.cli.download('es_core_news_sm')
nlp_es = spacy.load('es_core_news_sm')

# Gráfica de los tokens más utilizados para Spam
Palabras_Spam = {}

for clase, mensaje in zip(df2['clase'], df2['mensaje']):
    if clase == 'Spam':
        for token in nlp_es(mensaje):
            if(Palabras_Spam.get(token.text) == None):
                Palabras_Spam.setdefault(token.text, 1)
            else:
                Palabras_Spam[token.text] += 1

Palabras_Spam2 = {}
for clave, valor in zip(Palabras_Spam.keys(), Palabras_Spam.values()):
    if(valor >= 5):
        Palabras_Spam2.setdefault(clave, valor)

plt.figure(figsize=(10,6))
plt.bar(Palabras_Spam2.keys(), Palabras_Spam2.values())
plt.title('Palabras más importantes para Spam')
plt.xticks(rotation=80)
plt.show()

In [ ]:
# Gráfica de los tokens más utilizados para No Spam
Palabras_NoSpam = {}

for clase, mensaje in zip(df2['clase'], df2['mensaje']):
    if clase == 'NoSpam':
        for token in nlp_es(mensaje):
            if(Palabras_NoSpam.get(token.text) == None):
                Palabras_NoSpam.setdefault(token.text, 1)
            else:
                Palabras_NoSpam[token.text] += 1

Palabras_NoSpam2 = {}
for clave, valor in zip(Palabras_NoSpam.keys(), Palabras_NoSpam.values()):
    if(valor >= 2):
        Palabras_NoSpam2.setdefault(clave, valor)

plt.figure(figsize=(10,6))
plt.bar(Palabras_NoSpam2.keys(), Palabras_NoSpam2.values())
plt.title('Palabras más importantes para NoSpam')
plt.xticks(rotation=80)
plt.show()

## Paso05 - Construcción del modelo de NLP

In [ ]:
# Separar la información (Dataset) en conjuntos de entrenamiento y validación

# Importar la librería de Sklearn
from sklearn.model_selection import train_test_split

# Definimos nuestra información (Nombre de la clase (Y) y valor (X))
X = df2['mensaje'] # Entrada (Información que tenemos)
Y = df2['clase'] # Salidas (Respuestas deseadas para cada texto de entrada)

# Segmentamos la información en conjuntos de entrenamiento y de validación (80 / 20)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [ ]:
# Se importa la librería para extraccion de características y vectorización
# de los ejemplos usando la función CountVectorizer()
# (Permite conocer información de los daros de entrenamiento)
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

#  Con esta línea se construye un diccionario (Vocabulario de palabras) y se
# cuenta el número de palabras que hay para cada elemento del diccionario
X_train_counts = count_vect.fit_transform(X_train)

# (Shape) El primer número representa los mensajes, y el segundo el vocabulario usado
X_train_counts.shape

In [ ]:
# Se crea un Pipeline para asignar la secuencia de procesos
from sklearn._____ import Pipeline
from sklearn._____.text import TfidfVectorizer
from sklearn._____ import LinearSVC

# En el Pipeline se mete en un arreglo la secuencia de pasos que se desea
# seguir, o elementos que se desea enviar, en este caso 1) Vector de Tf-idf
# 2) El modelo LinearSVC
# En esta línea se hace la vectorización y se ejecuta el clasificador en un solo paso
clasificador_Texto = Pipeline([('tfidf', TfidfVectorizer()),('clf', LinearSVC())])

In [ ]:
# Visualizamos los parámetros de nuestro modelo (De referencia)
print(clasificador_Texto.get_params().keys())

# Podemos cambiar algunos de los parámetros como por ejempplo, uso de minúsculas
# filtrado de stopwords y máximo de características a usar
clasificador_Texto.set_params(tfidf__lowercase=True, tfidf__stop_words=['de', 'para'], tfidf__max_features=None)

In [ ]:
clasificador_Texto.fit(_____, _____) # Entrasas VS Salidas
Predicciones = clasificador_Texto.predict(_____) # Usar conjunto de pruebas
print(X_test)
print(Predicciones)

In [ ]:
clasificador_Texto.predict(["Gracias por cofirmar tu asistencia",
                            "Hola usuario, tenemos una oferta para tí",
                            "Solo por hoy, el precio ha bajado",
                            "Esperamos que te encuentras bien, ya hemos hecho la activación de tu cuenta",
                            "Porqué no revisas estos ofertones??",
                            "Mantequilla"])

## Matriz de confusión

In [ ]:
# Matriz de confusión y Métricas de evaluación del modelo
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

# Impresión de matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(_____, Predicciones))

# Impresión de procentaje de Accuracy del modelo
print("\nAccuracy del modelo: ")
print(metrics.accuracy_score(_____, Predicciones))

# Impresión de las métricas para el modelo
print("\nMétricas de evaluación:")
print(classification_report(_____, Predicciones))